In [ ]:
import requests
from bs4 import BeautifulSoup as soup

In [ ]:
import csv
import os

In [ ]:
os.getcwd()
os.chdir("TripAdvReviews")

In [ ]:
os.getcwd()

In [ ]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.45 Safari/537.36 OPR/82.0.4227.23'}

In [ ]:
rootUrl = 'https://www.tripadvisor.com/Hotels-g293734-oa120-Marrakech_Marrakech_Safi-Hotels.html'
html = requests.get(rootUrl, headers = headers)
html.status_code

In [ ]:
bsobj = soup(html.content,'lxml')

# get links to reviews

In [ ]:
links = []
for review in bsobj.findAll('a', {'class' : 'review_count'}):
    a = review['href']
    a = "https://www.tripadvisor.com" + a 
    print(a)
    print(a.find("Reviews"))
    a = a[:(a.find("Reviews")+7)] + '-or{}' + a[(a.find("Reviews")+7):]
    print(a)
    links.append(a)
    
links

In [ ]:
len(links)

# function to get number of review per hotel

In [ ]:
def reviewsNumber(target_link):
    html0 = requests.get(target_link.format(5), headers = headers)
    sleep(randint(1,5))
    bsobj0 = soup(html0.content, 'lxml')
    listLang = []
    nbReview = bsobj0.findAll('span', {'class': 'fwSIg'})
    for nbr in nbReview:
        listLang.append(nbr.text)
    #get index of English
    ind = listLang.index('English')
    #get reviews number
    nbReviews = bsobj0.findAll('span', {'class': 'cvxmR'})
    #substring
    nbr = nbReviews[ind].text[1:]
    nbr = nbr[:-1]
    nbr  =nbr.replace(',','')
    nbr = int(nbr)
    return nbr

# get reviews

In [ ]:
from random import randint
from time import sleep
from requests.exceptions import ConnectionError

#open csv file 
file = open('ReviewsMarrakech.csv', 'w', encoding='UTF-8')
writer = csv.writer(file)
#set file header
header = ["hotel_name", "city", "rate", "review","reviewer_name" ,"reviewer_country", "stay_date"]
writer.writerow(header)

for link in links:
    print(link)
    maxReviews = reviewsNumber(link)
    d = [j for j in range(5,maxReviews,5)]

    #fill csv file
    for i in d:
        #catch connectionExcpetion
        try:
            html2 = requests.get(link.format(i), headers = headers)
        except ConnectionError as e:
            continue
        
        sleep(randint(1,5))
        
        bsobj2 = soup(html2.content, 'lxml')
        #find hotel name
        hotel_name = bsobj2.find('h1', {'id':'HEADING'})
        if hotel_name:
            hotel_name = hotel_name.text
        else:
            hotel_name = bsobj2.find('h1').text            
        #find hotel city
        H_city = bsobj2.findAll('li', {'class', 'breadcrumb'})[3].span.text
        #find reviews
        for r in bsobj2.findAll('div', {'class' : 'cWwQK'}):
            #user
            user = r.find('a',{'class','ui_header_link'}).text
            #user origin
            user_country = r.find('span',{'class','ShLyt'})
            if user_country:
                user_country = user_country.text.strip()
                if user_country.find(',') != -1:
                    user_country = user_country[user_country.find(',')+2:]
            else:
                user_country = ''
            #rate
            rate = r.find('span', {'class': 'ui_bubble_rating'})
            if rate:
                rate = float(rate['class'][1][7:])/10
            else:
                rate = -1
            #review
            rev = r.find('q').span.text.strip()
            #date of stay
            stay_date = r.find('span', {'class','euPKI'})
            if stay_date:

                stay_date = r.find('span', {'class','euPKI'}).text
                stay_date = stay_date[14:]
            else:
                stay_date = ''

            row = [ hotel_name,H_city,rate, rev, user, user_country, stay_date ]
            #print(row)
            writer.writerow(row)
file.close()